Proyecto: Crecimiento Automático de Datos (V2)
En esta primera parte de la V2, el objetivo es que mi base de datos crezca sola de forma inteligente. En lugar de agregar información repetida, el sistema ahora es capaz de revisar qué locales ya conoce y agregar solamente los nuevos que vayan apareciendo.

Para lograr esto, uní dos de mis proyectos anteriores usando la tecnología que instalé en mi Raspberry Pi 4:

El Cerebro del Proyecto: Uso el servidor que armé en mi proyecto de Monitoreo de Temperatura. Gracias a que instalé Docker, este servidor (una Raspberry Pi de bajo consumo) está encendido las 24 horas para que el sistema nunca deje de buscar información.

La Memoria (Base de Datos): Uso la misma base de datos (PostgreSQL) del proyecto anterior, pero le agregué secciones nuevas ("tablas") para guardar esta nueva información de locales de forma organizada.

El Robot de Tareas (n8n): Creé un nuevo flujo de trabajo automatizado. Su tarea es simple: buscar datos, comparar con lo que ya tenemos y guardar solo lo que sea nuevo.

En resumen: Aproveché la infraestructura que ya tenía funcionando en casa para crear un sistema que se mantiene actualizado por sí solo, ahorrando tiempo y evitando errores.

Para gestionar y ver mis datos de forma fácil, utilizo DBeaver (una herramienta que funciona como una "ventana" hacia mi base de datos).

Lo primero que hice fue conectar este servicio a la base de datos que vive dentro de mi Raspberry Pi. Esto me permite enviarle órdenes al servidor desde mi propia computadora sin complicaciones. (Ver imagen parte 1).

Una vez establecida la conexión, procedí a crear una nueva tabla utilizando el siguiente código:

In [ ]:
CREATE TABLE prospectos_mercado (
    id SERIAL PRIMARY KEY,
    nombre VARCHAR(255),
    comuna VARCHAR(100),
    direccion TEXT,
    telefono VARCHAR(50),
    categoria VARCHAR(100),
    fecha_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

Luego, le agregué una condición especial a la tabla para que la base de datos sea más inteligente. El objetivo es que niegue automáticamente el ingreso de cualquier dato que esté repetido.

Para lograr esto, configuré el campo "nombre" como un filtro obligatorio. De esta manera, si el sistema intenta registrar un local que ya existe, la base de datos lo identifica y lo rechaza, asegurando que nuestra información esté siempre limpia y sin duplicados.

In [ ]:
ALTER TABLE prospectos_mercado 
ADD CONSTRAINT nombre_unico UNIQUE (nombre);

Con estos pasos, ya tenemos la conexión lista y podemos visualizar la tabla fácilmente. Para encontrarla dentro de DBeaver, solo debemos seguir esta ruta:

Nombre de la base de datos > Schemas > public > Tables

Ahí debería aparecer la tabla que creamos.

Nota técnica: Si el código se ejecutó correctamente pero no ves la tabla en la lista, simplemente haz clic derecho y selecciona "Refresh" (Actualizar) para que el programa refresque la vista y te la muestre.(Ver imagen parte 1)

Configuración del Flujo en n8n
Para este proceso, utilizaremos la misma cuenta local configurada en el proyecto anterior. A continuación, explico el funcionamiento general del flujo y luego detallaremos cada paso.

Estructura del Workflow
Nodo de Cron (Programación): Define la frecuencia con la que se ejecutará el flujo de forma automática (por ejemplo, una vez al día, a la semana o al mes).

Nodo de Código (Generación de datos): Este nodo entrega un listado con las 6 comunas que queremos investigar. Genera 6 elementos (items) de salida, uno por cada comuna.

Nodo Loop (Bucle): Es el corazón del ciclo. Toma una comuna, realiza todo el proceso y, al terminar, vuelve a empezar con la siguiente hasta completar las 6 vueltas.

Nodo de Búsqueda (Google Maps): Utiliza una API (requiere llave de acceso) para realizar búsquedas en la web como si fuera una persona. En este caso, busca "Pescaderías en [Comuna], Santiago de Chile" y entrega los resultados encontrados.

Nodo Split (Desglose): Toma el resultado anterior y lo separa por local_results. Esto transforma un solo bloque de búsqueda en 15 locales individuales (15 items).

Nodo de Limpieza (Eliminar duplicados): Filtra los resultados para no procesar locales repetidos que ensuciarían la tabla o causarían errores en el sistema.

Nodo de Base de Datos (PostgreSQL): Inserta los valores en nuestra tabla de la Raspberry Pi.

Consejo: Mapea solo las columnas obligatorias. Si algún dato (como una URL) no existe, el sistema lo dejará como "nulo" sin detenerse.

Nota Importante: En las opciones del nodo, activamos "Skip on Conflict". Esto permite que, si la base de datos rechaza un nombre repetido, el flujo continúe sin bloquearse.

Nodo Wait (Espera): Agregué una pausa de 5 minutos entre cada ciclo. Esto sirve para simular un comportamiento humano y evitar que Google nos detecte y bloquee como si fuéramos un robot (bot).

Codigo del nodo de codigo

In [ ]:
return [
  { comuna: "La Florida" },
  { comuna: "Macul" },
  { comuna: "Peñalolén" },
  { comuna: "Ñuñoa" },
  { comuna: "Providencia" },
  { comuna: "Las Condes" }
];

en imagenes parte 1 habran pantallazo de las configuraciones y el flujo